In [1]:
#Install Packages
!pip install faiss-cpu
!pip install sentence-transformers

In [2]:
# import necessary libraries
import pandas as pd
pd.set_option('display.max_colwidth', 100)

In [3]:
df = pd.read_csv("sample_text.csv")
df.shape

(8, 2)

In [4]:
df

,text,category
0,Meditation and yoga can improve mental health,Health
1,"Fruits, whole grains and vegetables helps control blood pressure",Health
2,These are the latest fashion trends for this week,Fashion
3,Vibrant color jeans for male are becoming a trend,Fashion
4,The concert starts at 7 PM tonight,Event
5,Navaratri dandiya program at Expo center in Mumbai this october,Event
6,Exciting vacation destinations for your next trip,Travel
7,Maldives and Srilanka are gaining popularity in terms of low budget vacation places,Travel


Step 1 : Create Source embeddings for the text column

In [5]:
from sentence_transformers import SentenceTransformer

c:\Users\Jay\AppData\Local\Programs\Python\Python311\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [6]:
encoder = SentenceTransformer("all-mpnet-base-v2")
vectors = encoder.encode(df.text)

c:\Users\Jay\AppData\Local\Programs\Python\Python311\Lib\site-packages\huggingface_hub\file_download.py:140: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\Jay\.cache\huggingface\hub\models--sentence-transformers--all-mpnet-base-v2. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to activate developer mode, see this article: https://docs.microsoft.com/en-us/windows/apps/get-started/enable-your-device-for-development
  warnings.warn(message)


In [7]:
vectors.shape

(8, 768)

In [8]:
dim = vectors.shape[1]
dim

768

step 2: Build a FAISS index for vectors

In [9]:
import faiss

index = faiss.IndexFlatL2(dim)

step 3 : normalize the source vectors (as we are using L2 distance to measure similarity) and add to the index

In [10]:
index.add(vectors)

In [11]:
index

<faiss.swigfaiss_avx2.IndexFlatL2; proxy of <Swig Object of type 'faiss::IndexFlatL2 *' at 0x0000022543AE7930> >

step 4 : encode search text using same encoder and normalize the ouput vector

In [12]:
search_query = "I want to buy a polo t-shirt"
# search_query = "looking for places to visit during the holidays"
# search_query = "An apple a day keeps the doctor away"
vec = encoder.encode(search_query)
vec.shape

(768,)

In [13]:
import numpy as np
svec = np.array(vec).reshape(1,-1)
svec.shape

(1, 768)

In [14]:
faiss.normalize_L2(svec)

step 5 : search for similar vector in the FAISS index created

In [16]:

distances, I = index.search(svec, k=2)
distances

array([[1.3844837, 1.4039096]], dtype=float32)

In [17]:
I

array([[3, 2]], dtype=int64)

In [18]:
df.loc[[3,2]]

,text,category
3,Vibrant color jeans for male are becoming a trend,Fashion
2,These are the latest fashion trends for this week,Fashion


In [19]:
df.loc[I[0]]

,text,category
3,Vibrant color jeans for male are becoming a trend,Fashion
2,These are the latest fashion trends for this week,Fashion


In [21]:
I.tolist()

[[3, 2]]

In [22]:
search_query

'I want to buy a polo t-shirt'

In [23]:
row_indices = I.tolist()[0]
row_indices

[3, 2]

In [24]:
df.loc[row_indices]

,text,category
3,Vibrant color jeans for male are becoming a trend,Fashion
2,These are the latest fashion trends for this week,Fashion


In [25]:
search_query

'I want to buy a polo t-shirt'